# 1 대 1 대응으로 train과 train_aws합치기

In [156]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import interpolate
%matplotlib inline

TRAIN_PATH = 'C:/Users/js/Desktop/AISPRAK/사회문제 해결형/data/TRAIN/'
TRAIN_AWS_PATH = 'C:/Users/js/Desktop/AISPRAK/사회문제 해결형/data/TRAIN_AWS/'
META_AWS_PATH = 'C:/Users/js/Desktop/AISPRAK/사회문제 해결형/data/META/awsmap.csv'
META_PM_PATH = 'C:/Users/js/Desktop/AISPRAK/사회문제 해결형/data/META/pmmap.csv'

LINEAR_PATH = 'C:/Users/js/Desktop/AISPRAK/사회문제 해결형/data/TRAIN_AWS_LINEAR/'
POLY_PATH = 'C:/Users/js/Desktop/AISPRAK/사회문제 해결형/data/TRAIN_AWS_POLYNOMIAL_2/'

TRAIN_LINEAR_PATH = 'C:/Users/js/Desktop/AISPRAK/사회문제 해결형/data/TRAIN_LINEAR/'
TRAIN_POLY_PATH = 'C:/Users/js/Desktop/AISPRAK/사회문제 해결형/data/TRAIN_POLYNOMIAL/'

TRAIN_MERGE_LINEAR_PATH = 'C:/Users/js/Desktop/AISPRAK/사회문제 해결형/data/MERGE_LINEAR/'
TRAIN_MERGE_POLYNOMIAL_PATH = 'C:/Users/js/Desktop/AISPRAK/사회문제 해결형/data/MERGE_POLYNOMIAL/'

mpl.rcParams['figure.figsize'] = (20,10)
mpl.rcParams['axes.grid'] = False

In [157]:
# 데이터 Null 처리를 위해 보간법(Interpolation) 적용
# Linear,Lagrange, Newton Spline, Cubic, Polynomial(2,3차)까지 적용해보자.

# https://www.d3view.com/interpolation-methods-for-time-series-data/
# https://towardsdatascience.com/polynomial-interpolation-3463ea4b63dd

In [183]:
# df = pd.read_csv(TRAIN_AWS_PATH+'공주.csv')
# df.rename(columns={'연도': 'year', '일시' : 'date', "지점": 'place','기온(°C)':'temp','풍향(deg)':'wind_direction','풍속(m/s)':'wind_speed','강수량(mm)':'rain','습도(%)':'humid'},inplace=True)
# df.tail()

In [184]:
# df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M')

In [186]:
# Linear Interpolation

for aws in os.listdir(TRAIN_PATH):
    df = pd.read_csv(TRAIN_PATH+aws)
    df.rename(columns={'연도': 'year', '일시' : 'date', "측정소": 'place'},inplace=True)
    df['year'] += 2001
    df['date'] = df['year'].astype('str') + "-" + df['date'].astype('str')
    df['year'] = df['year'].astype('object')
    df['PM2.5'] = df['PM2.5'].interpolate(method='linear')
    df.to_csv(TRAIN_LINEAR_PATH+aws,index=False)

In [187]:
# Linear Interpolation

for aws in os.listdir(TRAIN_PATH):
    df = pd.read_csv(TRAIN_PATH+aws)
    df.rename(columns={'연도': 'year', '일시' : 'date', "측정소": 'place'},inplace=True)
    df['year'] += 2001
    df['date'] = df['year'].astype('str') + "-" + df['date'].astype('str')
    df['year'] = df['year'].astype('object')
    df['PM2.5'] = df['PM2.5'].interpolate(method='polynomial',order=2)
    df.to_csv(TRAIN_POLY_PATH+aws,index=False)

In [188]:
# Linear Interpolation

for aws in os.listdir(TRAIN_AWS_PATH):
    df = pd.read_csv(TRAIN_AWS_PATH+aws)
    df.rename(columns={'연도': 'year', '일시' : 'date', "지점": 'place','기온(°C)':'temp','풍향(deg)':'wind_direction','풍속(m/s)':'wind_speed','강수량(mm)':'rain','습도(%)':'humid'},inplace=True)
    df['year'] += 2001
    df['date'] = df['year'].astype('str') + "-" + df['date'].astype('str')
    df['year'] = df['year'].astype('object')
    df[['temp','wind_direction','wind_speed','rain','humid']] = df[['temp','wind_direction','wind_speed','rain','humid']].interpolate(method='linear')
    df.to_csv(LINEAR_PATH+aws,index=False)

In [189]:
# Polynomial Interpolation

for aws in os.listdir(TRAIN_AWS_PATH):
    df = pd.read_csv(TRAIN_AWS_PATH+aws)
    df.rename(columns={'연도': 'year', '일시' : 'date', "지점": 'place','기온(°C)':'temp','풍향(deg)':'wind_direction','풍속(m/s)':'wind_speed','강수량(mm)':'rain','습도(%)':'humid'},inplace=True)
    df['year'] += 2001
    df['date'] = df['year'].astype('str') + "-" + df['date'].astype('str')
    df['year'] = df['year'].astype('object')
    df[['temp','wind_direction','wind_speed','rain','humid']] = df[['temp','wind_direction','wind_speed','rain','humid']].interpolate(method='polynomial',order=2)
    df.to_csv(POLY_PATH+aws,index=False)

# 데이터 TRAIN과 TRAIN_AWS 합치기

In [199]:
one2one_col = {'정림동' : '오월드', '문창동' : '오월드',  '읍내동' : '장동',  '노은동' : '계룡',  '아름동' : '세종고운',
                '신흥동' : '세종연서',  '공주' : '공주',  '논산' : '논산',  '대천2동' : '대천항',  '홍성읍' : '홍북',
                '예산군' : '예산',  '모종동' : '아산',  '신방동' : '성거',  '성성동' : '성거',  '동문동' : '태안',
                '이원면' : '안도',  '독곶리' : '대산'}



In [197]:
for train in os.listdir(TRAIN_LINEAR_PATH):
    df = pd.read_csv(TRAIN_LINEAR_PATH+train)
    
    merge_df = pd.read_csv(LINEAR_PATH+one2one_col[train.split('.')[0]]+'.csv')
    merged_df = pd.merge(df,merge_df,how='inner',on='date')

    merged_df.to_csv(TRAIN_MERGE_LINEAR_PATH+train,index=False)

In [198]:
for train in os.listdir(TRAIN_POLY_PATH):
    df = pd.read_csv(TRAIN_POLY_PATH+train)
    
    merge_df = pd.read_csv(POLY_PATH+one2one_col[train.split('.')[0]]+'.csv')
    merged_df = pd.merge(df,merge_df,how='inner',on='date')

    merged_df.to_csv(TRAIN_MERGE_POLYNOMIAL_PATH+train,index=False)